In [ ]:
import glob
import os
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from swap_strategy import create_qaoa_swap_circuit, SATMapper
from utils import load_graph_from_npy
from graph_to_ansatz import GraphAnsatzConverter
from qiskit.transpiler.passes.routing.commuting_2q_gate_routing import SwapStrategy
import yaml
from qiskit import QuantumCircuit
from rl.game import Game, encode_state
from rl.network import ResNet

base_path = "graphs"
index = "20241130"
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)
swap_strategy_results = defaultdict(list)

In [ ]:
def plot_swap_counts(results, label=None, color="blue"):
    keys = list(results.keys())
    means = [np.mean(values) for values in results.values()]
    std_devs = [np.std(values) for values in results.values()]

    plt.figure(figsize=(8, 6))
    plt.plot(keys, means, label=label, marker="o", linestyle="-", color=color)
    plt.fill_between(
        keys,
        [m - s for m, s in zip(means, std_devs)],
        [m + s for m, s in zip(means, std_devs)],
        color=color,
        alpha=0.2,
    )

    plt.xlabel("Qubits")
    plt.ylabel("Swap counts")
    plt.title("Swap counts vs. qubit counts")
    plt.legend()
    plt.grid(True)

    plt.show()

In [ ]:
for n in range(4, 11):
    pattern = os.path.join(base_path, f"adj_matrix_{n}_*.npy")
    file_paths = glob.glob(pattern)
    for file_path in file_paths:
        g = load_graph_from_npy(file_path)
        swap_strategy = SwapStrategy.from_line(range(len(g.nodes())))
        sm = SATMapper(timeout=10)
        remapped_graph, edge_map, min_swap_layers = sm.remap_graph_with_sat(
            graph=g, swap_strategy=swap_strategy, max_layers=1
        )
        # print("Map from old to new nodes: ", edge_map)
        # print("Min SWAP layers:", min_swap_layers)
        converter = GraphAnsatzConverter(
            g if remapped_graph is None else remapped_graph
        )
        qaoa_circ = create_qaoa_swap_circuit(
            converter.hamiltonian, swap_strategy, qaoa_layers=1
        )
        depth = qaoa_circ.decompose(reps=1).depth(lambda instr: instr.name == "swap")
        # qaoa_circ.decompose(reps=1).draw(output="mpl")
        # swap_strategy_results[n].append(
        #     len(qaoa_circ.decompose(reps=1).get_instructions("swap"))
        # )
        
        swap_strategy_results[n].append(depth)

In [ ]:
plot_swap_counts(swap_strategy_results, label="swap_strategy")

In [ ]:
ai_results = defaultdict(list)

In [ ]:
import tensorflow as tf
from rl.game import Game, encode_state
from rl.network import ResNet

# qubits = config["game_settings"]["N"]
for qubits in range(6, 7):
    game = Game(qubits, config)
    pattern = os.path.join(base_path, f"adj_matrix_{qubits}_*.npy")
    file_paths = glob.glob(pattern)
    if qubits == 4:
        training_steps = 200
    elif qubits == 5:
        training_steps = 500
    elif qubits == 6:
        training_steps = 250
    elif qubits == 7:
        training_steps = 700
    else:
        training_steps = 700
    network = ResNet(action_space=len(game.coupling_map),config=config)
    network.load_weights(f"checkpoints/network{qubits}_{index}_{training_steps}.weights.h5")

    for file_path in file_paths:
        trial = 0
        min_depth = float("inf")
        while trial < 10:
            state = np.load(file_path)
            ans = []
            done = False
            total_score = 0
            step_count = 0
            prev_action = None
            while not done and step_count < 50:
                encoded_state = encode_state(state, qubits)
                input_state = np.expand_dims(encoded_state, axis=0)
                policy_output, value_output = network.predict(input_state)
                policy = policy_output[0]
                if prev_action is not None:
                    indices = [i for i in range(len(game.coupling_map)) if i != prev_action]
                    prob = policy[indices]
                    if prob.sum() < 1e-9:
                        action = np.random.choice(indices)
                    else:
                        action = np.random.choice(indices, p=prob / prob.sum())
                else:
                    indices = list(range(len(game.coupling_map)))
                    action = np.random.choice(indices, p=policy)
                selected_action = game.coupling_map[action]
                # print(f"Step {step_count}: Selected action {selected_action}")
                ans.append(selected_action)
                # アクションの適用
                state, done, _ = game.step(state, action, prev_action)
                prev_action = action
                step_count += 1
            qc = QuantumCircuit(qubits)
            for swap in ans:
                qc.swap(*swap)
            depth = qc.depth(lambda instr: instr.name == "swap")
            min_depth = min(depth, min_depth)
            trial += 1
        ai_results[qubits].append(min_depth)
        # ai_results[qubits].append(len(ans))
ai_results

In [6]:
ai_swap_depth = defaultdict(list)
ai_swap_cnt = defaultdict(list)
qubits = 6
game = Game(qubits,config)
network = ResNet(action_space=len(game.coupling_map),config=config)
network.load_weights(f"checkpoints/network{qubits}_{index}_300.weights.h5")

def evaluate_self_play(qubits, network):
    pattern = os.path.join(base_path, f"adj_matrix_{qubits}_*.npy")
    file_paths = glob.glob(pattern)
    avg_depth = []
    avg_counts = []
    for file_path in file_paths:
        state = np.load(file_path)
        trial = 0
        min_depth = float("inf")
        while trial < 15:
            swap_pairs = []
            done = False
            total_score = 0
            step_count = 0
            prev_action = None
            while not done and step_count < game.MAX_STEPS:
                encoded_state = encode_state(state, qubits)
                input_state = np.expand_dims(encoded_state, axis=0)
                policy_output, value_output = network.predict(input_state)
                policy = policy_output[0]
                if prev_action is not None:
                    indices = [i for i in range(len(game.coupling_map)) if i != prev_action]
                    prob = policy[indices]
                    if prob.sum() < 1e-9:
                        action = np.random.choice(indices)
                    else:
                        action = np.random.choice(indices, p=prob / prob.sum())
                else:
                    indices = list(range(len(game.coupling_map)))
                    action = np.random.choice(indices, p=policy)
                selected_action = game.coupling_map[action]
                swap_pairs.append(selected_action)
                state, done, _ = game.step(state, action, prev_action)
                prev_action = action
                step_count += 1
            qc = QuantumCircuit(qubits)
            if not done:
                depth = game.MAX_STEPS
                swap_count = game.MAX_STEPS
            else:
                for swap in swap_pairs:
                    qc.swap(*swap)
                depth = qc.depth(lambda instr: instr.name == "swap")
                swap_count = len(swap_pairs)
            if min_depth > depth:
                min_depth = depth
                min_swap_count = swap_count
            trial += 1
        ai_swap_cnt[qubits].append(min_swap_count)
        ai_swap_depth[qubits].append(min_depth)
    return np.mean(ai_swap_depth[qubits]), np.mean(ai_swap_cnt[qubits])
evaluate_self_play(qubits,network)

(21.8, 21.833333333333332)

In [7]:
ai_swap_depth

defaultdict(list,
            {6: [1,
              40,
              40,
              40,
              40,
              1,
              1,
              1,
              40,
              1,
              40,
              40,
              1,
              1,
              1,
              40,
              40,
              1,
              40,
              1,
              1,
              40,
              40,
              1,
              40,
              40,
              1,
              40,
              1,
              40]})

In [ ]:
ai_results = {5: [2,
              2,
              3,
              2,
              3,
              3,
              2,
              3,
              2,
              4,
              2,
              2,
              2,
              2,
              2,
              2,
              2,
              2,
              2,
              2,
              2,
              1,
              3,
              2,
              2,
              2,
              2,
              2,
              2,
              3],
             6: [3,
              2,
              3,
              3,
              4,
              3,
              2,
              6,
              4,
              4,
              6,
              5,
              2,
              2,
              2,
              3,
              4,
              2,
              5,
              1,
              5,
              4,
              4,
              6,
              3,
              4,
              3,
              4,
              3,
              3],
             7: [4,
              3,
              5,
              4,
              3,
              5,
              4,
              3,
              4,
              4,
              3,
              5,
              3,
              6,
              3,
              4,
              5,
              4,
              4,
              5,
              3,
              5,
              6,
              4,
              4,
              5,
              5,
              4,
              3,
              4],
             8: [8,
              6,
              7,
              6,
              7,
              5,
              5,
              5,
              6,
              8,
              8,
              5,
              7,
              6,
              6,
              5,
              5,
              5,
              5,
              6,
              4,
              6,
              6,
              4,
              7,
              4,
              6,
              5,
              4,
              5],
             9: [5,
              7,
              8,
              10,
              10,
              10,
              6,
              5,
              4,
              9,
              7,
              7,
              6,
              8,
              8,
              7,
              8,
              8,
              8,
              7,
              7,
              9,
              5,
              6,
              3,
              7,
              9,
              7,
              10,
              6],
             10: [9,
              12,
              11,
              9,
              10,
              10,
              9,
              9,
              12,
              12,
              12,
              9,
              11,
              8,
              12,
              12,
              13,
              11,
              10,
              12,
              13,
              9,
              12,
              12,
              11,
              9,
              12,
              11,
              13,
              11]}

In [ ]:
plot_swap_counts(ai_results, label="ai_strategy")

In [ ]:
def plot_swap_counts_combined(*results_list, labels=None, colors=None):

    plt.figure(figsize=(8, 6))
    for i, results in enumerate(results_list):
        label = labels[i] if labels and i < len(labels) else f"Strategy {i+1}"
        color = colors[i] if colors and i < len(colors) else None
        keys = list(results.keys())
        means = [np.mean(values) for values in results.values()]
        std_devs = [np.std(values) for values in results.values()]
        plt.plot(keys, means, label=label, marker="o", linestyle="-", color=color)
        plt.fill_between(
            keys,
            [m - s for m, s in zip(means, std_devs)],
            [m + s for m, s in zip(means, std_devs)],
            color=color,
            alpha=0.2,
        )
    plt.xlabel("Qubits")
    plt.ylabel("depth")
    plt.title("Depth vs. qubit counts")
    plt.legend()
    plt.grid(True)
    plt.show()


plot_swap_counts_combined(
    swap_strategy_results,
    ai_results,
    labels=["swap_strategy", "ai_strategy"],
    colors=["blue", "red"],
)

In [ ]:
import numpy as np
import tensorflow as tf
import yaml
from rl.network import ResNet

from rl.game import Game
qubits =4
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)
game = Game(qubits, config)
network = ResNet(action_space=len(game.coupling_map),config=config)

# network = tf.keras.models.load_model(f"checkpoints/network4_20241120_50.keras")
network.load_weights("checkpoints/network4_20241120_50.weights.h5")